# Create tokenizer & embedding matrix
This notebook generates a tokenizer and embedding matrix based on medical articles from the Dutch Wikipedia. Everything in this notebook is based on the MedCAT tutorials, see https://github.com/CogStack/MedCAT/tree/master/tutorial.

In [1]:
import numpy as np
from medcat.tokenizers.meta_cat_tokenizers import ByteLevelBPETokenizer, TokenizerWrapperBPE
from gensim.models import Word2Vec
from pathlib import Path

In [2]:
# Input
data_dir = Path.cwd().parents[0] / 'data'
text_file = data_dir / 'nlwiki_20200901_geneeskunde_depth4.txt'

# Output
model_dir = Path.cwd().parents[0] / 'models' / 'bilstm'
model_dir.mkdir(parents=True, exist_ok=True)
embeddings_file = model_dir / 'embeddings.npy'

## Create tokenizer

In [3]:
# Create, train and save the tokenizer
tokenizer = ByteLevelBPETokenizer(add_prefix_space=True)
tokenizer.train(text_file.__str__())

In [4]:
# Convert to wrapped tokenizer for MetaCAT
wrapped_tokenizer = TokenizerWrapperBPE(hf_tokenizers=tokenizer)

# Save the tokenizer
wrapped_tokenizer.save(model_dir.__str__())

## Create embedding matrix

In [5]:
# Tokenize text and train with Word2Vec
text_data = []
with open(text_file, encoding='utf-8') as text:
    for line in text:
        text_data.append(tokenizer.encode(line).tokens)
w2v = Word2Vec(text_data, vector_size=300, min_count=1)

In [6]:
# Check trained word2vec model
# Ġ denotes start of word (a space)
w2v.wv.most_similar('Ġhoesten')

[('Ġkortademigheid', 0.912898063659668),
 ('Ġbraken', 0.8875458240509033),
 ('Ġjeuk', 0.8771429657936096),
 ('Ġniezen', 0.8712943196296692),
 ('Ġspierpijn', 0.8658164143562317),
 ('Ġvermoeidheid', 0.8626760244369507),
 ('Ġmisselijkheid', 0.8601147532463074),
 ('Ġovergeven', 0.8595143556594849),
 ('Ġzweten', 0.8576074242591858),
 ('Ġdiarree', 0.8571567535400391)]

In [7]:
# Create embedding matrix
embeddings = []
for i in range(tokenizer.get_vocab_size()):
    word = tokenizer.id_to_token(i)
    if word in w2v.wv:
        embeddings.append(w2v.wv[word])
    else:
        # Assign a random vector if the word was not frequent enough to receive an embedding
        embeddings.append(np.random.rand(300))
# Save the embeddings
np.save(open(embeddings_file, 'wb'), np.array(embeddings))